# Packages and libraries

In [33]:
!pip install numpy pandas 

In [34]:
import pandas as pd
import datetime as dt

## Data Scrapping

### 1. Output Gap Data extraction

First we got the quarterly GDP for the period [Office for National Statistics](https://www.ons.gov.uk/economy/grossdomesticproductgdp/timeseries/ybha/qna)

WE got the yearly output gap [Office for Budget Responsibility (OBR)](https://obr.uk/public-finances-databank-2024-25/)

Using the quarterly estimates developed [OBR: Output gap measurement: judgement and uncertainty](https://obr.uk/docs/dlm_uploads/WorkingPaperNo5.pdf) 



In [37]:
# Use the raw URL from the GitHub repository
xlsx_url = "https://raw.githubusercontent.com/guri99uy/ST449_Project/52611de9d475e711c4c917c4d5ca137427404612/outputgap.xlsx"

df_outputgap = pd.read_excel(xlsx_url, engine='openpyxl') 

# Aux function for formatting
def parse_qqyyyy(qqyyyy):
    quarter = int(qqyyyy[1])
    year = int(qqyyyy[2:])
    # Map the quarter to the first month of that quarter
    quarter_start_month = {1: 1, 2: 4, 3: 7, 4: 10}
    month = quarter_start_month[quarter]

    return dt.datetime(year, month, 1)


df_outputgap['QQYYYY'] = df_outputgap['QQYYYY'].apply(parse_qqyyyy)
df_outputgap.rename(columns={'QQYYYY': 'Date'}, inplace=True)

#Get Date in Quarters
df_outputgap['Date'] = pd.to_datetime(df_outputgap['Date'])
df_outputgap['Quarter'] = df_outputgap['Date'].dt.to_period('Q')
df_outputgap = df_outputgap.drop(columns=['Date'])
df_outputgap['GDP_Pot (m£)'] = df_outputgap['GDP_Pot (m£)'].round(0).astype(int)
df_outputgap['Output_gap (%)'] = df_outputgap['Output_gap (%)'].round(2)
print(df_outputgap.head())

   GDP_Real (m£)  GDP_Pot (m£)  Output_gap (%) Quarter
0         127119        130233            2.45  1987Q3
1         129815        133288            2.68  1987Q4
2         133283        137215            2.95  1988Q1
3         136630        141576            3.62  1988Q2
4         140801        145602            3.41  1988Q3


### 2. Interest Rate
Got .xlsx file from [Bank of Engalnd](https://www.bankofengland.co.uk/boeapps/database/Bank-Rate.asp)

Lets process the data to: 
1. Get the quarter average
2. Assign missing quarters with the last value

In [39]:
# Use the raw URL from the GitHub repository
url = "https://raw.githubusercontent.com/guri99uy/ST449_Project/7715079b32be2ea0b9e2e77a3f7b81244f85720f/Bank_Rate.xlsx"
df_interest_rate = pd.read_excel(url, engine='openpyxl')
df_interest_rate.columns = ['Date', 'Interest_rate']

# Aux function for formatting
def parse_date(date_str):
    # Handle the format '07 Nov 24'
    return dt.datetime.strptime(date_str, '%d %b %y')

df_interest_rate['Date'] = df_interest_rate['Date'].apply(parse_date)


if df_interest_rate['Interest_rate'].dtype == 'object':
    df_interest_rate['Interest_rate'] = df_interest_rate['Rate'].str.replace(',', '.').astype(float)
else:
    df_interest_rate['Interest_rate'] = pd.to_numeric(df_interest_rate['Interest_rate'], errors='coerce')

df_interest_rate['Date'] = pd.to_datetime(df_interest_rate['Date'])
df_interest_rate['Quarter'] = df_interest_rate['Date'].dt.to_period('Q')

# Group by the Quarter
quarterly_avg_rate = (
    df_interest_rate.groupby('Quarter', as_index=False)['Interest_rate']
    .mean()
    .rename(columns={'Interest_rate': 'Avg_Interest_Rate'})
)

full_quarters = pd.period_range('1975Q1', '2007Q4', freq='Q')
quarterly_avg_rate['Quarter'] = pd.PeriodIndex(quarterly_avg_rate['Quarter'], freq='Q')
quarterly_avg_rate = quarterly_avg_rate.set_index('Quarter').reindex(full_quarters)

# Fill missing values with the value from the previous quarter
quarterly_avg_rate['Avg_Interest_Rate'] = quarterly_avg_rate['Avg_Interest_Rate'].ffill()
quarterly_avg_rate.reset_index(inplace=True)
quarterly_avg_rate.rename(columns={'index': 'Quarter'}, inplace=True)

Quarterly_interest_rates = quarterly_avg_rate[
    (quarterly_avg_rate['Quarter'] >= '1987Q3') & (quarterly_avg_rate['Quarter'] <= '2007Q4')
]
Quarterly_interest_rates.reset_index(inplace=True)
Quarterly_interest_rates = Quarterly_interest_rates.drop(columns=['index'])
print(Quarterly_interest_rates.head())

  Quarter  Avg_Interest_Rate
0  1987Q3              9.880
1  1987Q4              8.880
2  1988Q1              8.630
3  1988Q2              8.080
4  1988Q3             10.755


### 3. Inflation



In [41]:
# GitHub raw URL for inflation
url = "https://raw.githubusercontent.com/guri99uy/ST449_Project/a1a15126e784af7897ac4f44fbead11867582aa5/inf_Data.csv"
inflation = pd.read_csv(url)
inf_data = inflation.rename(columns={"Implied GDP deflator at market prices: SA Index": "GDP Deflator"})
inf_data.rename(columns={"Title": "Quarter"}, inplace=True)
inf_data["Quarter"] = inf_data["Quarter"].str.replace(r"(\d{4})\sQ(\d)", r"\1Q\2", regex=True)

# Compute the inflation rate as the percentage change in the GDP Deflator
inf_data['Inflation_Rate (%)'] = inf_data['GDP Deflator'].pct_change() * 100
inf_data['Inflation_Rate (%)'] = inf_data['Inflation_Rate (%)'].round(2)

# Remove the first row with Quarter value '1987Q2'
inf_data = inf_data[inf_data['Quarter'] != '1987Q2'].reset_index(drop=True)

print(inf_data.head())

  Quarter  GDP Deflator  Inflation_Rate (%)
0  1987Q3       35.8724                0.20
1  1987Q4       36.2206                0.97
2  1988Q1       36.5950                1.03
3  1988Q2       37.3205                1.98
4  1988Q3       37.9849                1.78


### 4. Merge relevant data
1. Output Gap
2. Interest rate
3. Inflation
   

In [43]:
# Format all df
Quarterly_interest_rates['Quarter'] = pd.PeriodIndex(Quarterly_interest_rates['Quarter'], freq='Q')
df_outputgap['Quarter'] = pd.PeriodIndex(df_outputgap['Quarter'], freq='Q')
inf_data['Quarter'] = pd.PeriodIndex(inf_data['Quarter'], freq='Q')

# Merge the datasets
merged_df = pd.merge(Quarterly_interest_rates, df_outputgap, on='Quarter', how='inner')  # Inner join
merged_df = pd.merge(merged_df, inf_data, on='Quarter', how='inner')  # Inner join

# Display the merged DataFrame
print(merged_df.head())



  Quarter  Avg_Interest_Rate  GDP_Real (m£)  GDP_Pot (m£)  Output_gap (%)  \
0  1987Q3              9.880         127119        130233            2.45   
1  1987Q4              8.880         129815        133288            2.68   
2  1988Q1              8.630         133283        137215            2.95   
3  1988Q2              8.080         136630        141576            3.62   
4  1988Q3             10.755         140801        145602            3.41   

   GDP Deflator  Inflation_Rate (%)  
0       35.8724                0.20  
1       36.2206                0.97  
2       36.5950                1.03  
3       37.3205                1.98  
4       37.9849                1.78  


### 5. Save Clean Data

In [45]:
data_folder = 'Data'

# Create the 'Data' folder 
os.makedirs(data_folder, exist_ok=True)  # `exist_ok=True` avoids error if folder already exists


output_file = os.path.join(data_folder, 'Clean_data.csv')
merged_df.to_csv(output_file, index=False)

print(f"\nMerged data has been saved to '{output_file}'")


Merged data has been saved to 'Data\Clean_data.csv'
